# Compute Centerline Angles

* Read in eig_basis
* Compute centerline angles for new worms
* Project new centerline angles onto eig_basis
* Export new centerline angles and projections


In [ ]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.interpolate import interpn
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.io as sio
%pylab inline

In [ ]:
# load data
basis_directory = '../Data/' # for Katherine's computer
#basis_directory = 'Data/' # for Nick's computer
basis_file_name = 'eig_basis'
basis_file = sio.loadmat(basis_directory+basis_file_name)

# Read eig_basis
These eigenbases were computed previously from Ashley's single worm optogenetic experiments.

We're going to compare them later to SVDs from the new dataset of worms.

In [ ]:
# elements of eig_basis: eig_basis
eig_basis = np.array(basis_file['eig_basis'])
(dim, num_basis) = np.shape(eig_basis)

f = plt.figure(figsize=(20,5*2))
ax = plt.axes()
for i in range(1,num_basis):
    ax.plot(eig_basis[:,i])

# Read centerline data

Read data and clean bad frames

In [ ]:
# Read in position data
# Read in worm data from SharedData .npz file

Worm = 6#'GFP'
worm_data = np.load('../SharedData/Worm'+str(Worm)+'.npz')
print('The loaded npz contains the variables:\n', np.sort([i for i in worm_data]))

rec_x = worm_data['Center_x']
rec_y = worm_data['Center_y']
rec_time = worm_data['CenterTimes']

[center_length, center_time] = np.shape(rec_x)

# Clean up + remove bad frames
if Worm == 5: # Bad frames at 905
    rec_x = np.delete(rec_x, 905, 1)
    rec_y = np.delete(rec_y, 905, 1)
    rec_time = np.delete(rec_time, 905, 0)
    

# Convert position data to centerline representation

Here we follow the methods outlined in Stephens 2008. To find the curvature of the worm (K) we'd like to take the second derivative, but in practice this is noisey. Instead we compute the angle between the tangentline to the worm and the horizontal, then noramlize the angle by subtracting out the mean. 

There's an additionally processing of wrap around for angles before we normalize. 

In [ ]:
# Compute tangent lines
tangent_x = diff(rec_x,axis=0)
tangent_y = diff(rec_y,axis=0)

# Compute the angle between the tangent and the horizontal
angles = np.arctan2(tangent_y,tangent_x)

(points_t, time_t) = shape(angles)

# Getting rid of the wrap around for pi 
for t in range(0,time_t):
    for p in range(0,points_t-1):
        if (angles[p,t]-angles[p+1,t]) < -3:
            angles[p+1,t] = angles[p+1,t]-2*np.pi
        elif (angles[p,t]-angles[p+1,t]) > 3:
            angles[p+1,t] = angles[p+1,t]+2*np.pi

            
# Subtract out the average angle to normalize
theta = angles-np.mean(angles,axis=0)


In [ ]:
print(shape(rec_x))
print(shape(np.diff(rec_x,axis=0)))
print(tangent_x[0,1])
print(tangent_x[len(tangent_x)-1,1])

In [ ]:
# Test graph to make sure everything worked as expected
f = plt.figure(figsize=(20,10))
ax = plt.axes()
ax.plot(theta[:,0:10])
ax.set_title("Test plot of 10 computed centerline angles")

# Project new centerline angles onto old eigenworms
* Find representation of new centerline data with 6 eigenworms
* Export data

In [ ]:
# Project new centerline thetas onto old worms
proj_theta = np.empty([time_t,num_basis])
for t in range(0,time_t):
    for b in range(0,num_basis):
        proj_theta[t,b] = dot(theta[:,t], eig_basis[:,b]) 

f = plt.figure(figsize=(20,10))
ax = plt.axes()
ax.plot(proj_theta[0:10,:])
ax.set_title("Plot of projections of 10 centerlines onto 6 eigenworms")

# Match centerline data and neural data

Adapted from Animate Centerline

In [ ]:
G = worm_data['G']
[neuron_length, neuron_time] = np.shape(G)

# Make sure centerline times encompass neuron times
if rec_time[0] > Time[0] or rec_time[-1] < Time[-1]:
    viable_indices = np.array([rec_time[0] < i and rec_time[-1] > i for i in Time])
    Time = Time[viable_indices]
    print('Neuron data was trimmed to only data with corresponding centerline info. \\
          Data now starts at ' + str(Time[0]) + 'sec and ends at ' + str(Time[-1]) + 'sec')
    
ind = np.zeros(len(Time), dtype=int)

i = 0
j = 0
for i in range(len(Time)):
    while rec_time[j] < Time[i]:
        j += 1
    if abs(rec_time[j-1] - Time[i]) < abs(rec_time[j] - Time[i]):
        ind[i] = j-1
    else:
        ind[i] = j
        
# Check
print('The average difference in time courses is ' + 
     '%.1f' % (np.average(abs(rec_time[ind] - Time))*1000) + 'msec and the max difference is ' +
     '%.1f' % (np.max(abs(rec_time[ind] - Time))*1000) + 'msec')

proj_neural_thetas = proj_theta[ind]

## Export

Exports all the same information as Consolodate, except the G and R here have NaN's stripped out

In [ ]:
file_name = '../SharedData/Worm_Angles/WormAngle'+str(Worm)
print(file_name)
print(Worm)

In [ ]:
# export
file_name = '../SharedData/WormAngle'+str(Worm)
print(file_name)
ethoTrack = worm_data['ethoTrack']
Time = worm_data['Time']
NPos = worm_data['NPos']
G_Raw = worm_data['G_Raw']
R_Raw = worm_data['R_Raw']
CenterTimes = worm_data['CenterTimes']
Center_x = worm_data['Center_x']
Center_y = worm_data['Center_y']
G = worm_data['G']
R = worm_data['R']
if Worm=='GFP':
    G_photocorr = worm_data['G_photocorr']
    R_photocorr = worm_data['R_photocorr']
    np.savez_compressed(file_name, 
        G=G, R=R, ethoTrack=ethoTrack, Time=Time, NPos=NPos,
        G_Raw=G_Raw, R_Raw=R_Raw, G_photocorr = G_photocorr, R_photocorr = R_photocorr,
        CenterTimes=CenterTimes, Center_x=Center_x, Center_y=Center_y,
        thetas = theta, proj_thetas = proj_theta, proj_neural_thetas = proj_neural_thetas
                   )
else:
    np.savez_compressed(file_name, 
        G=G, R=R, ethoTrack=ethoTrack, Time=Time, NPos=NPos,
        G_Raw=G_Raw, R_Raw=R_Raw,
        CenterTimes=CenterTimes, Center_x=Center_x, Center_y=Center_y,
        thetas = theta, proj_thetas = proj_theta, proj_neural_thetas = proj_neural_thetas
                   )

In [ ]:
ethoTrack = worm_data['ethoTrack']
Time = worm_data['Time']
NPos = worm_data['NPos']
G_Raw = worm_data['G_Raw']
R_Raw = worm_data['R_Raw']
CenterTimes = worm_data['CenterTimes']
Center_x = worm_data['Center_x']
Center_y = worm_data['Center_y']
G = worm_data['G']
R = worm_data['R']
print(shape(G_Raw))
print(shape(G))